1. why p-1?
2. 是否需要存储公钥私钥然后再进行环签名
3. 改成DSA lib中的公私钥加密和随机数生成

In [3]:
import random
from hashlib import sha256

def generate_keys(g, p):
    # 生成私钥s
    s = random.randint(1, p-2)
    # 计算公钥v = g^s mod p
    v = pow(g, s, p)
    return s, v

def schnorr_sign(message, g, p, s):
    # 选择随机数k
    k = random.randint(1, p-2)
    # 计算r = g^k mod p
    r = pow(g, k, p)
    # 将消息和r的值哈希，然后与私钥s结合产生签名
    hash_value = int(sha256((str(r) + message).encode()).hexdigest(), 16)
    x = (k - s * hash_value) % (p-1)
    return r, x

def schnorr_verify(message, g, p, v, r, x):
    # 重新计算r的哈希值
    hash_value = int(sha256((str(r) + message).encode()).hexdigest(), 16)
    # 使用公钥v和签名x、r计算验证值
    rv = (pow(g, x, p) * pow(v, hash_value, p)) % p
    return rv == r

# 设置大素数p和其原根g
p = 1000000007  # 一个大素数
g = 5  # p的原根

# 生成密钥
s, v = generate_keys(g, p)

# 签名消息
message = "Hello, Schnorr!"
r, x = schnorr_sign(message, g, p, s)
print(f"Signature: (r={r}, x={x})")

# 验证签名
valid = schnorr_verify(message, g, p, v, r, x)
print(f"Signature valid: {valid}")


Signature: (r=265444478, x=82644123)
Signature valid: True


In [9]:
import random 
from hashlib import sha256


p = 1000000007  # prime
g = 5  # primitive root of p

def generate_keys(g, p):
    # secret key s
    s = random.randint(1, p-2)
    # public key = g^s mod p
    v = pow(g, s, p)
    return s, v

msg = 'Hello, Ring Signature'
print('msg:', msg)

x1, y1 = generate_keys(g, p)
x2, y2 = generate_keys(g, p)
x3, y3 = generate_keys(g, p)
public_key_list = [y1, y2, y3]
print('pubkeys:', public_key_list)
Y = ''.join([str(i) for i in public_key_list])
print('Y:', Y)

r, R = generate_keys(g, p)
e3 = int(sha256((str(msg) + str(Y) + str(pow(g, r, p))).encode()).hexdigest(), 16)
print('e3:', e3)

_, random_s3 = generate_keys(g, p)
print('random_s3:', random_s3)

calculation = (pow(g, random_s3, p)*pow(y3, e3, p)) % p
e1 = int(sha256((str(msg) + str(Y) + str(calculation)).encode()).hexdigest(), 16)
print('e1:', e1)

_, random_s1 = generate_keys(g, p)
print('random_s1:', random_s1)

calculation = (pow(g, random_s1, p)*pow(y1, e1, p)) % p
e2 = int(sha256((str(msg) + str(Y) + str(calculation)).encode()).hexdigest(), 16)
print('e2:', e2)

# why p-1?
s2 = (r - x2 * e2) % (p-1)
print('s2:', s2)

total_res = {
    'e1': e1,
    's1': random_s1,
    's2': s2,
    's3': random_s3
}

print('signature process total result:')
display(total_res)

msg: Hello, Ring Signature
pubkeys: [970258900, 777029397, 933394293]
Y: 970258900777029397933394293
e3: 22191167905611136301493664267411008641582145064466280289849370121915051925536
random_s3: 477113826
e1: 42765869882594708546099090092886709355147325619887755232369243277055749879857
random_s1: 686017743
e2: 65956070662949614275446208887963577650386801499139889911558385879645306034461
s2: 561637826
signature process total result:


{'e1': 42765869882594708546099090092886709355147325619887755232369243277055749879857,
 's1': 686017743,
 's2': 561637826,
 's3': 477113826}

In [8]:
calculation = (pow(g, total_res['s1'], p)*pow(y1, total_res['e1'], p)) % p
e2_V = int(sha256((str(msg) + str(Y) + str(calculation)).encode()).hexdigest(), 16)
calculation = (pow(g, total_res['s2'], p) * pow(y2, e2_V, p)) % p
e3_V = int(sha256((str(msg) + str(Y) + str(calculation)).encode()).hexdigest(), 16)
calculation = (pow(g, total_res['s3'], p)*pow(y3, e3_V, p)) % p
e1_V = int(sha256((str(msg) + str(Y) + str(calculation)).encode()).hexdigest(), 16)

print(e1_V)
print(e2_V)
print(e3_V)
print(e1_V==e1)
print(e2_V==e2)
print(e3_V==e3)

44456565993761453790862949750767730671395059856482746927246370380347071995958
62198233590465971368264658523262533127367827117784103704209069325293156887745
84077409024182565851783537121728300577596305538399313666557585127284400775226
True
True
True


In [57]:
e3_calculation = (pow(g, total_res['s2'], p) * pow(y2, e2_V, p)) % p
e3_calculation

188462739

In [58]:
e3_original_calculation = pow(g, r, p) % p
e3_original_calculation

188462739

In [79]:
(pow(g, total_res['s2'], p) * pow(y2, e2_V, p)) % p

135642210

In [80]:
pow(g, r, p)

135642210

In [1]:
import random
from hashlib import sha256

p = 1000000007  # prime
g = 5  # primitive root of p

def generate_keys(g, p):
    # secret key s
    s = random.randint(1, p-2)
    # public key = g^s mod p
    v = pow(g, s, p)
    return s, v

def generate_tag(x, g, p):
    # Generate tag using private key x and generator g
    tag = pow(g, x, p)
    return tag

msg = 'Hello, Ring Signature'
print('msg:', msg)

x1, y1 = generate_keys(g, p)
x2, y2 = generate_keys(g, p)
x3, y3 = generate_keys(g, p)
public_key_list = [y1, y2, y3]
print('pubkeys:', public_key_list)
Y = ''.join([str(i) for i in public_key_list])
print('Y:', Y)

tag = generate_tag(x1, g, p)  # Use x1 for the tag, can be any x
print('tag:', tag)

r, R = generate_keys(g, p)
e3 = int(sha256((str(msg) + str(Y) + str(tag) + str(pow(g, r, p))).encode()).hexdigest(), 16)
print('e3:', e3)

_, random_s3 = generate_keys(g, p)
print('random_s3:', random_s3)

calculation = (pow(g, random_s3, p)*pow(y3, e3, p)) % p
e1 = int(sha256((str(msg) + str(Y) + str(tag) + str(calculation)).encode()).hexdigest(), 16)
print('e1:', e1)

_, random_s1 = generate_keys(g, p)
print('random_s1:', random_s1)

calculation = (pow(g, random_s1, p)*pow(y1, e1, p)) % p
e2 = int(sha256((str(msg) + str(Y) + str(tag) + str(calculation)).encode()).hexdigest(), 16)
print('e2:', e2)

# why p-1?
s2 = (r - x2 * e2) % (p-1)
print('s2:', s2)

total_res = {
    'e1': e1,
    's1': random_s1,
    's2': s2,
    's3': random_s3,
    'tag': tag
}

print('signature process total result:')
display(total_res)


msg: Hello, Ring Signature
pubkeys: [245358855, 391265439, 619627151]
Y: 245358855391265439619627151
tag: 245358855
e3: 105715649303279684431592308991260205531123464133782301885974415331465511870586
random_s3: 526384682
e1: 19361389055214049803525966289360120734917675619618798464103609142787164445316
random_s1: 895878476
e2: 112739571309604336382124080772522420737509100379681067348133901259730899812673
s2: 237649469
signature process total result:


{'e1': 19361389055214049803525966289360120734917675619618798464103609142787164445316,
 's1': 895878476,
 's2': 237649469,
 's3': 526384682,
 'tag': 245358855}

In [2]:
calculation = (pow(g, total_res['s1'], p)*pow(y1, total_res['e1'], p)) % p
e2_V = int(sha256((str(msg) + str(Y) + str(tag) + str(calculation)).encode()).hexdigest(), 16)
calculation = (pow(g, total_res['s2'], p) * pow(y2, e2_V, p)) % p
e3_V = int(sha256((str(msg) + str(Y) + str(tag) + str(calculation)).encode()).hexdigest(), 16)
calculation = (pow(g, total_res['s3'], p)*pow(y3, e3_V, p)) % p
e1_V = int(sha256((str(msg) + str(Y) + str(tag) + str(calculation)).encode()).hexdigest(), 16)

print(e1_V)
print(e2_V)
print(e3_V)
print(e1_V==e1)
print(e2_V==e2)
print(e3_V==e3)

19361389055214049803525966289360120734917675619618798464103609142787164445316
112739571309604336382124080772522420737509100379681067348133901259730899812673
105715649303279684431592308991260205531123464133782301885974415331465511870586
True
True
True
